In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 3.3 ms


In [4]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 47.1 ms


In [5]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 5.66 ms


In [6]:
#Get all the frames from S3
#start with 355
url_list = []
for i in range(355,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

time: 3.53 ms


In [ ]:
# Face Detection & Emotioin Analysis 
emotions = {}
count = 355
#355

for single_image in url_list:

    json = { 'url': single_image } 
    emotions[count] = detect(single_image)
    processed = processRequest(json, data, headers, params)
    
    try:
        emotions[count].append(processed[0])  
    except:
        pass 

    if count < 5350:
        count += 5
    
    time.sleep(3)

In [ ]:
# Face Detection & Emotioin Analysis - UPDATED
processed = {}
face_detection = {}
count = 355
#355

for single_image in url_list:
 
    face_detection[count] = detect(single_image)
    
    json = { 'url': single_image } 
    processed[count] = processRequest(json, data, headers, params)    
    
    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 5
    
    time.sleep(3)

In [ ]:
# Save the result as Json
import json
with open('debate1_full.json', 'w') as fp:
    json.dump(emotions, fp, sort_keys=True)